In [ ]:
import os
from crnn import CRNN
import torch
import torch.nn.functional as F
import torchvision
import dataset
from torch.autograd import Variable
import utils
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nh = 100
alphabet = '0123456789'
nclass = len(alphabet) + 1
nc = 1
mynet = CRNN(32, nc, nclass, nh)

In [ ]:
converter = utils.strLabelConverter(alphabet)

In [ ]:
mynet.load_state_dict(torch.load('/home/sherlock/Documents/express_recognition/data/model_save/telephone/newbaseline.pth'))
mynet = mynet.cuda()
mynet = mynet.eval()

In [ ]:
root = '/home/sherlock/Documents/express_recognition/data/validation/'
img_root = root + 'telephone_data_2'
txt_root = root + 'telephone_label_validation_2.txt'

In [ ]:
dset = dataset.Dataset(img_root=img_root, txt_root=txt_root)

In [ ]:
print('All Valid Image number is {}'.format(len(dset)))

In [ ]:
def get_acc():
    transformer = dataset.resizeNormalize((256, 32))
    num_correct = 0
    for i, img_path in enumerate(dset.image):
        image = Image.open(img_path).convert('L')
        image = transformer(image).cuda()
        image = image.view(1, *image.size())
        image = Variable(image)

        preds = mynet(image) # sequence_length x batch_size x n_classes
        _, preds = preds.max(2) # sequence_length x batch_size x 1
        preds = preds.view(-1) # sequence_length x 1
        preds_size = torch.IntTensor([preds.size(0)])
        raw_pred = converter.decode(preds.data, preds_size, raw=True)
        sim_pred = converter.decode(preds.data, preds_size, raw=False)
        if sim_pred == dset.label[i]:
            num_correct += 1
        else:
            pic = Image.open(img_path)
            plt.imshow(pic)
            plt.show()
            print(img_path.split('/')[-1])
            print('{} => {}'.format(raw_pred, sim_pred))
    print('total number is: {}'.format(num_correct))
    print('total accuracy: {:.6f}'.format(num_correct/len(dset)))

In [ ]:
def decode(x):
    i = 0
    text = ''
    while i < len(x)-1:
        temp = x.cpu().data[i]
        if temp == 0:
            i += 1
        else:
            text += str(temp-1)
            if temp == x.cpu().data[i+1]:
                i += 2
            else:
                i += 1
    if i == len(x)-1:
        if x.cpu().data[i] != 0:
            text += str(x.cpu().data[i]-1)
    return text

In [ ]:
def get_acc_new():
    transformer = dataset.resizeNormalize((256, 32))
    num_correct = 0
    new_correct = 0
    for j, img_path in enumerate(dset.image):
        image = Image.open(img_path).convert('L')
        image = transformer(image).cuda()
        image = image.view(1, *image.size())
        image = Variable(image)

        preds = mynet(image) # sequence_length x batch_size x n_classes
        _, preds = preds.max(2) # sequence_length x batch_size x 1
        preds = preds.view(-1) # sequence_length x 1
        preds_size = torch.IntTensor([preds.size(0)])
        raw_pred = converter.decode(preds.data, preds_size, raw=True)
        sim_pred = converter.decode(preds.data, preds_size, raw=False)
        my_pred = decode(preds)
        if sim_pred == dset.label[j]:
            num_correct += 1
        else:
            print('old error')
            pic = Image.open(img_path)
            plt.imshow(pic)
            plt.show()
            print('{} => {}'.format(raw_pred, sim_pred))
        if my_pred == dset.label[j]:
            new_correct += 1
        else:
            print('new error')
            pic = Image.open(img_path)
            plt.imshow(pic)
            plt.show()
            print('{} => {}'.format(raw_pred, my_pred))
        print('*'*10)
    print('old total number is: {}'.format(num_correct))
    print('new total number is: {}'.format(new_correct))
    print('old total accuracy: {:.6f}'.format(num_correct/len(dset)))
    print('new total accuracy: {:.6f}'.format(new_correct/len(dset)))

In [ ]:
get_acc_new()

In [ ]:
def show_result(img_name):
    root = '/home/sherlock/Documents/express_recognition/data/validation/telephone_data'
    img_path = os.path.join(root, img_name)
    transformer = dataset.resizeNormalize((256, 32))
    image = Image.open(img_path).convert('L')
    image = transformer(image).cuda()
    image = image.view(1, *image.size())
    image = Variable(image)
    
    mynet.eval()
    preds = mynet(image) # sequence_length x batch_size x n_classes
    _, preds = preds.max(2) # sequence_length x batch_size x 1
    preds = preds.view(-1) # sequence_length x 1
    preds_size = torch.IntTensor([preds.size(0)])
    raw_pred = converter.decode(preds.data, preds_size, raw=True)
    sim_pred = converter.decode(preds.data, preds_size, raw=False)
    pic = Image.open(img_path)
    plt.imshow(pic)
    plt.show()
    print('{} => {}'.format(raw_pred, sim_pred))

In [ ]:
show_result('1_10_0_0.png')

In [ ]:
root = '/home/sherlock/Documents/express_recognition/data/validation/telephone_data'
img_path = os.path.join(root, '1_10_0_0.png')
transformer = dataset.resizeNormalize((256, 32))
image = Image.open(img_path).convert('L')
image = transformer(image).cuda()
image = image.view(1, *image.size())
image = Variable(image)

mynet.eval()
preds = mynet(image) # preds sequence_length x batch_size x n_classes

In [ ]:
preds = preds.squeeze(1)

In [ ]:
preds = F.softmax(preds)
prob, label = torch.max(preds, 1)

In [ ]:
tele = ''
for i in range(label.size(0)):
    if prob[i].cpu().data[0] > 0.9:
        if label[i].cpu().data[0] == 0:
            tele += '-'
        else:
            tele += str(label[i].cpu().data[0]-1)

In [ ]:
for i in range(preds.size(0)):
    label = preds.cpu().data[i, :]
    label = label.numpy()
    plt.plot(label)
    plt.title('{}'.format(i))
    plt.show()

In [ ]:
all_file_name = os.listdir('/home/sherlock/Documents/express_recognition/data/validation/telephone/')

for img in all_file_name:
    show_result(img)
    print()